In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [134]:
data = pd.read_csv("data/unimelb_training.csv")
data.set_index("Grant.Application.ID",drop=True,inplace=True)
data.index.name=None

D:\Users\greg_\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (90,105,119,120,123,124,132,134,135,138,139,147,149,150,153,154,162,164,165,168,169,177,179,183,184,192,194,198,199,207,209,213,214,224,237,239,244) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [135]:
data = data.ix[:,:25]

In [136]:
data.head(2)

,Grant.Status,Sponsor.Code,Grant.Category.Code,Contract.Value.Band...see.note.A,Start.date,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,RFCD.Code.3,...,SEO.Code.1,SEO.Percentage.1,SEO.Code.2,SEO.Percentage.2,SEO.Code.3,SEO.Percentage.3,SEO.Code.4,SEO.Percentage.4,SEO.Code.5,SEO.Percentage.5
1,1,NaN,NaN,A,8/11/05,280199.0,100.0,0.0,0.0,0.0,...,700299.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2B,10A,B,11/11/05,280103.0,30.0,280106.0,30.0,280203.0,...,700103.0,50.0,700102.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0


In [137]:
# Create dummies for Sponsor.Code
data = pd.concat([data,pd.get_dummies(data["Sponsor.Code"],prefix="Sponsor")],axis=1)
data.drop("Sponsor.Code",axis=1,inplace=True)

In [138]:
# Create dummies for Grant.Category.Code
data = pd.concat([data,pd.get_dummies(data["Grant.Category.Code"],prefix="Cat")],axis=1)
data.drop("Grant.Category.Code",axis=1,inplace=True)

In [139]:
# Deal with date

a = pd.to_datetime(data["Start.date"])
year = pd.Series([date.year for date in a],name="year")
month = pd.Series([date.month for date in a],name="month")
day = pd.Series([date.day for date in a],name="day")
data["Start.year"] = year
data["Start.month"] = month
data["Start.day"] = day
data.drop(["Start.date"],axis=1,inplace=True)

In [140]:
# Create dummies for Contract.Val
data = pd.concat([data,pd.get_dummies(data["Contract.Value.Band...see.note.A"],prefix="Contract")],axis=1)
data.drop("Contract.Value.Band...see.note.A",axis=1,inplace=True)

In [141]:
# Create dummies for RFCD
sub1 = pd.DataFrame(data.ix[:,1:3])
sub1.rename(columns={"RFCD.Code.1":"RFCD.Code",
                     "RFCD.Percentage.1":"RFCD.Percentage"},inplace=True)

sub2 = pd.DataFrame(data.ix[:,3:5])
sub2.rename(columns={"RFCD.Code.2":"RFCD.Code",
                     "RFCD.Percentage.2":"RFCD.Percentage"},inplace=True)

sub3 = pd.DataFrame(data.ix[:,5:7])
sub3.rename(columns={"RFCD.Code.3":"RFCD.Code",
                     "RFCD.Percentage.3":"RFCD.Percentage"},inplace=True)

sub4 = pd.DataFrame(data.ix[:,7:9])
sub4.rename(columns={"RFCD.Code.4":"RFCD.Code",
                     "RFCD.Percentage.4":"RFCD.Percentage"},inplace=True)

sub5 = pd.DataFrame(data.ix[:,9:11])
sub5.rename(columns={"RFCD.Code.5":"RFCD.Code",
                     "RFCD.Percentage.5":"RFCD.Percentage"},inplace=True)

RFCD = sub1.append(sub2,ignore_index=False)
RFCD = RFCD.append(sub3,ignore_index=False)
RFCD = RFCD.append(sub4,ignore_index=False)
RFCD = RFCD.append(sub4,ignore_index=False)

RFCD[RFCD["RFCD.Code"].isnull()]=0

piv = pd.pivot_table(RFCD,columns="RFCD.Code",index=RFCD.index,fill_value=0,dropna=False)
piv[piv.isnull()]=0
piv.index.name=None

#data = pd.concat([data,piv],axis=1)
data.drop(["RFCD.Code.1","RFCD.Percentage.1",
           "RFCD.Code.2","RFCD.Percentage.2",
           "RFCD.Code.3","RFCD.Percentage.3",
           "RFCD.Code.4","RFCD.Percentage.4",
           "RFCD.Code.5","RFCD.Percentage.5",],axis=1,inplace=True)

In [142]:
# Create dummies for SEO
sub1 = pd.DataFrame(data.ix[:,1:3])
sub1.rename(columns={"SEO.Code.1":"SEO.Code",
                     "SEO.Percentage.1":"SEO.Percentage"},inplace=True)

sub2 = pd.DataFrame(data.ix[:,3:5])
sub2.rename(columns={"SEO.Code.2":"SEO.Code",
                     "SEO.Percentage.2":"SEO.Percentage"},inplace=True)

sub3 = pd.DataFrame(data.ix[:,5:7])
sub3.rename(columns={"SEO.Code.3":"SEO.Code",
                     "SEO.Percentage.3":"SEO.Percentage"},inplace=True)

sub4 = pd.DataFrame(data.ix[:,7:9])
sub4.rename(columns={"SEO.Code.4":"SEO.Code",
                     "SEO.Percentage.4":"SEO.Percentage"},inplace=True)

sub5 = pd.DataFrame(data.ix[:,9:11])
sub5.rename(columns={"SEO.Code.5":"SEO.Code",
                     "SEO.Percentage.5":"SEO.Percentage"},inplace=True)

SEO = sub1.append(sub2,ignore_index=False)
SEO = SEO.append(sub3,ignore_index=False)
SEO = SEO.append(sub4,ignore_index=False)
SEO = SEO.append(sub4,ignore_index=False)

SEO[SEO["SEO.Code"].isnull()]=0

piv = pd.pivot_table(SEO,columns="SEO.Code",index=RFCD.index,fill_value=0,dropna=False)
piv[piv.isnull()]=0
piv.index.name=None

#data = pd.concat([data,piv],axis=1)
data.drop(["SEO.Code.1","SEO.Percentage.1",
           "SEO.Code.2","SEO.Percentage.2",
           "SEO.Code.3","SEO.Percentage.3",
           "SEO.Code.4","SEO.Percentage.4",
           "SEO.Code.5","SEO.Percentage.5",],axis=1,inplace=True)

In [146]:
#from sklearn.preprocessing import LabelBinarizer

raw = pd.read_csv("data/unimelb_training.csv")

people = pd.DataFrame()
time_at_uni = pd.DataFrame()
for col in raw.columns:
    col_list = list(raw.columns) # list so I can use index later

    if col[:10] == 'Person.ID.':
        col_start = col_list.index(col)
        col_end = col_start + 15 # hard coded
        person = raw.iloc[:,col_start:col_end]

        # including our ID and status
        person.loc[:,'Grant.Application.ID'] = raw.loc[:,'Grant.Application.ID']
        person.loc[:,'Grant.Status'] = raw.loc[:,'Grant.Status']

        col_names = list(person.columns)
        col_names = [col.strip('0123456789') for col in col_names] # removes all numbers
        person.columns = col_names
        #print(len(person))

        # dealing with number of years at uni
        original = person.loc[:,'No..of.Years.in.Uni.at.Time.of.Grant.'].astype(str)
        transformed = [0 for _ in range(original.size)]
        transformed = np.where(original==">=0 to 5",2.5,transformed)
        transformed = np.where(original==">5 to 10",7.5,transformed)
        transformed = np.where(original==">10 to 15",12.5,transformed)
        transformed = np.where(original=="more than 15",20,transformed)
        transformed = pd.DataFrame(transformed)

        person["No..of.Years.in.Uni.at.Time.of.Grant."] = transformed
        #person = pd.concat([person, transformed], axis=1)
        people = people.append(person, ignore_index=True)

# summaries for team
num_ppl = len(people) / len(person) # should be 15!
people.index = people.loc[:,'Grant.Application.ID']

num_ppl = people.groupby(['Grant.Application.ID'])['Person.ID.'].count() # include

num_birth_years = people.groupby(['Grant.Application.ID'])['Year.of.Birth.'].count()
total_birth_year = people.groupby(['Grant.Application.ID'])['Year.of.Birth.'].sum()
avg_birth_year = total_birth_year / num_birth_years

total_birth_year = people.groupby(['Grant.Application.ID'])['Year.of.Birth.'].sum()
avg_birth_year = pd.Series(total_birth_year / num_ppl,name="Birth.Mean") # include

std_birth_year = pd.Series(people.groupby(['Grant.Application.ID'])['Year.of.Birth.'].std(),name="Birth.std")
std_birth_year[std_birth_year.isnull()]=0
min_birth_year = pd.Series(people.groupby(['Grant.Application.ID'])['Year.of.Birth.'].min(),name="Birth.min")
max_birth_year = pd.Series(people.groupby(['Grant.Application.ID'])['Year.of.Birth.'].max(),name="Birth.max")

avg_no_years_uni = pd.Series(people.groupby(['Grant.Application.ID'])['No..of.Years.in.Uni.at.Time.of.Grant.'].mean(),name="Years.Uni.avg")
std_no_years_uni = pd.Series(people.groupby(['Grant.Application.ID'])['No..of.Years.in.Uni.at.Time.of.Grant.'].std(),name="Years.Uni.std")
std_no_years_uni[std_no_years_uni.isnull()]=0
min_no_years_uni = pd.Series(people.groupby(['Grant.Application.ID'])['No..of.Years.in.Uni.at.Time.of.Grant.'].min(),name="Years.Uni.min")
max_no_years_uni = pd.Series(people.groupby(['Grant.Application.ID'])['No..of.Years.in.Uni.at.Time.of.Grant.'].max(),name="Years.Uni.max")


num_successful = pd.Series(people.groupby(['Grant.Application.ID'])['Number.of.Successful.Grant.'].sum(),name="num_succ")
num_unsuccessful = pd.Series(people.groupby(['Grant.Application.ID'])['Number.of.Unsuccessful.Grant.'].sum(),name="num_unsucc")
ratio_successful = pd.Series(num_successful/(num_successful+num_unsuccessful),name="Ratio.Successful")
ratio_successful[ratio_successful.isnull()]=0

# number of phds - TODO could be improved
num_phd  = pd.Series(people.groupby(['Grant.Application.ID'])['With.PHD.'].count(),name="phd.count") # assuming only value ever entered is yes # include
mean_phd  = pd.Series(people.groupby(['Grant.Application.ID'])['With.PHD.'].count() / \
    len(people.groupby(['Grant.Application.ID'])['With.PHD.']),name="phd.mean")# assuming only value ever entered is yes # include


# Mysterious variables
a__1 = pd.Series(people.groupby(['Grant.Application.ID'])['A..'].sum(),name="a..1.sum") 
a_1 = pd.Series(people.groupby(['Grant.Application.ID'])['A.'].sum(),name="a.1.sum") 
b_1 = pd.Series(people.groupby(['Grant.Application.ID'])['B.'].sum(),name="b.1.sum")
c_1 = pd.Series(people.groupby(['Grant.Application.ID'])['C.'].sum(),name="c.1.sum")
avg_a__1 = pd.Series(people.groupby(['Grant.Application.ID'])['A..'].mean(),name="a..1.avg")  
avg_a_1 = pd.Series(people.groupby(['Grant.Application.ID'])['A.'].mean(),name="a.1.avg")  
avg_b_1 = pd.Series(people.groupby(['Grant.Application.ID'])['B.'].mean(),name="b..1.avg")  
avg_c_1 = pd.Series(people.groupby(['Grant.Application.ID'])['C.'].mean(),name="c..1.avg")  

# Faculty
#fac = pd.Series(people.groupby(['Grant.Applicaiton.ID'])['Faculty.No..1'].mode(),name="faculty")

# creating DF with summary statistics for each person
people_summary = pd.DataFrame()
ppl_groupby = people.groupby(['Person.ID.'])
people_summary.loc[:,'Num applications'] = ppl_groupby['Grant.Application.ID'].count()
people_summary.loc[:,'Num grant status'] = ppl_groupby['Grant.Status'].sum()
people_summary.loc[:,'Num successful'] = ppl_groupby['Number.of.Successful.Grant.'].sum()
people_summary.loc[:,'Num unsuccessful'] = ppl_groupby['Number.of.Unsuccessful.Grant.'].sum()
people_summary.loc[:,'Ratio'] = people_summary.loc[:,'Num successful'] / (people_summary.loc[:,'Num unsuccessful'] + people_summary.loc[:,'Num successful'])
people_summary.sort(columns=['Ratio'], ascending=False, inplace=True)

# Number of persons per role type
roles = people["Role."].reset_index().pivot_table(columns="Role.",index=people.index,aggfunc=len,fill_value=0)
roles.fillna(value=0,inplace=True)

D:\Users\greg_\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (90,105,119,120,123,124,132,134,135,138,139,147,149,150,153,154,162,164,165,168,169,177,179,183,184,192,194,198,199,207,209,213,214,224,237,239,244) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\Users\greg_\Anaconda3\lib\site-packages\ipykernel\__main__.py:93: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [147]:
# Number of people in the team
input = pd.concat([data,num_ppl],axis=1)

# Year of birth
input = pd.concat([input,avg_birth_year],axis=1)
input = pd.concat([input,std_birth_year],axis=1)
input = pd.concat([input,min_birth_year],axis=1)
input = pd.concat([input,max_birth_year],axis=1)

# Years in Uni 
input = pd.concat([input,avg_no_years_uni],axis=1)
input = pd.concat([input,std_no_years_uni],axis=1)
input = pd.concat([input,min_no_years_uni],axis=1)
input = pd.concat([input,max_no_years_uni],axis=1)

# Application success ratio
input = pd.concat([input,ratio_successful],axis=1)
input = pd.concat([input,num_successful],axis=1)
#input = pd.concat([input,num_unsuccessful],axis=1)

# PhDs in team
input = pd.concat([input,num_phd],axis=1)
input = pd.concat([input,mean_phd],axis=1)

# Roles in team
input = pd.concat([input,roles],axis=1)

# Mysterious variables
input = pd.concat([input,a__1],axis=1)
input = pd.concat([input,a_1],axis=1)
input = pd.concat([input,b_1],axis=1)
input = pd.concat([input,c_1],axis=1)
input = pd.concat([input,avg_a__1],axis=1)
input = pd.concat([input,avg_a_1],axis=1)
input = pd.concat([input,avg_b_1],axis=1)
input = pd.concat([input,avg_c_1],axis=1)

# Cleaning
input.columns.name = None

In [148]:
input.shape

(8708, 359)

In [ ]:
pd.

In [ ]:
#input.dropna(inplace=True)
input.fillna(value=0,inplace=True)
X = input.ix[:,1:]
y = input.ix[:,0]
X.shape

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=131078)

"""
rfc = RandomForestClassifier(n_jobs=-1,random_state=0)
params={
    "n_estimators":[8,10,12],
    "min_samples_split":[2,4,6],
    "min_samples_leaf":[1,2,4],
}

cv = GridSearchCV(rfc,param_grid=params,n_jobs=-1).fit(X_train,y_train)

"""
gbc = GradientBoostingClassifier(random_state=0)
params={
    "n_estimators":[500],
    "min_samples_split":[6],
    "min_samples_leaf":[2],
}

cv = GridSearchCV(gbc,param_grid=params,n_jobs=-1).fit(X_train,y_train)

print(cv.best_params_)
print(cv.best_score_)

In [ ]:
df = pd.DataFrame([X.columns,cv.best_estimator_.feature_importances_]).transpose()
df.columns=["name","score"]
df.sort(columns=["score"],ascending=False)[:50]

In [ ]:
y_pred =cv.predict(X_test)

In [ ]:
accuracy = np.mean(y_pred == y_test)
accuracy

In [ ]:
from sklearn.metrics import confusion_matrix
conmat = np.array(confusion_matrix(y_test, y_pred))
confusion = pd.DataFrame(conmat, index=['not_granted', 'granted'],columns=['predicted_not_granted', 'predicted_granted'])
confusion

In [ ]:
from sklearn.metrics import classification_report
cls_rep = classification_report(y_test, y_pred)
print(cls_rep)

In [ ]:
for i,n in enumerate(input):
    print(i,n)

In [ ]:
X_train["Years.Uni.max"]

NameError: name '__version__' is not defined